![SAR, InSAR, PolSAR, and banner](https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/blackboard-banner.png)

# ISCE3 Sentinel-1 RTC

Generate an OPERA-like RTC from a Sentinel-1 SLC using OPERA's S1 reader and RTC processor.

Please note that this code is for testing and experimentation. 

The output of this notebook is **not an OPERA standard product**.

- https://github.com/opera-adt/s1-reader
- https://github.com/opera-adt/RTC

This notebook will only run in Linux with the correct environment installed. To install the `isce3_rtc` environment, see the OSL Notebook repo:

- https://github.com/ASFOpenSARlab/opensarlab-envs

To run on a non-Linux system, build and run the dockerfile provided in the opera-adt/RTC repo: https://github.com/opera-adt/RTC/blob/main/Docker/Dockerfile

**Author**: Alex Lewandowski; Alaska Satellite Facility

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display
from os import getenv

notebookUrl = notebookUrl.value
user = str(getenv("JUPYTERHUB_USER"))
env = str(getenv("CONDA_PREFIX"))

if env == '':
    env = 'Python 3 (base)'
if env != '/home/jovyan/.local/envs/isce3_rtc':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "isce3_rtc" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env.split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "isce3_rtc" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "isce3_rtc" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

In [ ]:
from datetime import datetime
import getpass
from pathlib import Path
import re
import shutil
from urllib.parse import urljoin
from zipfile import ZipFile

import asf_search
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from bs4 import BeautifulSoup
import numpy as np
from osgeo import gdal
gdal.UseExceptions()
import requests
import rtc
import yaml
import ipywidgets

## Authenticate with asf_search and download and extract a Sentinel-1 scene

In [ ]:
username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf_search.ASFSession().auth_with_creds(username, password)
except asf_search.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

In [ ]:
scene = input("Enter a Sentinal-1 RTC Scene ID: ")

data_dir_path = Path.cwd()/scene
data_dir_path.mkdir(exist_ok=True)

## Download the Sentinel-1 orbit file

In [ ]:
def datetime_in_orbitfile_range(datetime_str, orbitfile_name):
    orbit_date_regex = "(?<=\w[0-9]{7}T[0-9]{6}_V)\w[0-9]{7}T[0-9]{6}_\w[0-9]{7}T[0-9]{6}(?=\.EOF)"
    orbit_dates = re.search(orbit_date_regex, orbitfile_name)
    if orbit_dates:
        orbit_dates = orbit_dates.group(0).split('_')
    else:
        return False
    return orbit_dates[0] < datetime_str < orbit_dates[1]

In [ ]:
platform = scene[:3] if scene[:3] in ["S1A", "S1B"] else None

if platform is None:
    raise Exception("Platform not found in scene name")

s1_date_regex = "\w[0-9]{7}T[0-9]{6}"
datetime_str = re.search(s1_date_regex, scene).group(0)

In [ ]:
orbit_index_url = "https://s1qc.asf.alaska.edu/aux_poeorb/"
soup = BeautifulSoup(requests.get(orbit_index_url).content, "html.parser")

orbitfile_links = list(soup.select('a'))
orbitfile_link = [l for l in orbitfile_links if platform in str(l) and datetime_in_orbitfile_range(datetime_str, str(l))][0]
orbitfile_name = orbitfile_link.attrs.get("href")

href = urljoin(orbit_index_url, orbitfile_name)

In [ ]:
orbitfile_path = data_dir_path/orbitfile_name
r = user_pass_session.get(href)
with open(orbitfile_path, 'wb') as f:
    f.write(r.content)

## Download the COP30 DEM

In [ ]:
extents = None
with asf_search.granule_search(scene)[0].remotezip(session=user_pass_session) as z:
    with z.open(f"{scene}.SAFE/manifest.safe", "r") as data:
        bs_data = BeautifulSoup(data, "xml")
        extents = bs_data.find_all('gml:coordinates')
        
extents = extents[0].string.split(' ')
extents = [[int(np.floor(float(x.split(',')[0]))), int(np.floor(float(x.split(',')[1])))] for x in extents]

lats = list(set([c[0] for c in extents]))
lats = list(range(min(lats), max(lats)+1))    
longs = list(set([c[1] for c in extents]))
longs = list(range(min(longs), max(longs)+1))

In [ ]:
dem_s3_paths = []

for lat in lats:
    for long in longs:
        if lat >= 0:
            lat_dir = "N"
        else:
            lat_dir = "S"
        if long >= 0:
            long_dir = "E"
        else:
            long_dir = "W"
        
        dem_s3_paths.append(f"Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM/Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM.tif")
dem_s3_paths

In [ ]:
dem_dir_path = data_dir_path/"dems"
dem_dir_path.mkdir(exist_ok=True)

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

bucket_name = "copernicus-dem-30m"
bucket = s3.Bucket(bucket_name)

for s3_path in dem_s3_paths:
    bucket.download_file(s3_path, dem_dir_path/s3_path.split('/')[1]) 

In [ ]:
dem_paths = ' '.join([str(p) for p in (dem_dir_path.glob("*DEM.tif"))])
dem_paths

In [ ]:
merged_dem_path = dem_dir_path/"merged_dem.tif"
!gdal_merge.py -n 0.0 -o $merged_dem_path $dem_paths

## Copy and edit a template rtc_s1.yaml 

This is the config file we will use to define parameters for RTC processing

In [ ]:
rtc_config_template_path = Path(rtc.__file__).parent/"defaults/rtc_s1.yaml"
rtc_config_path = data_dir_path/"rtc_s1.yaml"
shutil.copyfile(rtc_config_template_path, rtc_config_path)

In [ ]:
with open(rtc_config_path, 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
cfg = config['runconfig']['groups']

cfg['input_file_group']['safe_file_path'] = [str(data_dir_path/f"{scene}.zip")]
cfg['input_file_group']['orbit_file_path'] = [str(orbitfile_path)]
cfg['dynamic_ancillary_file_group']['dem_file'] = str(merged_dem_path)
cfg['dynamic_ancillary_file_group']['dem_description'] = f"Merged COP30 DEMS from s3://copernicus-dem-30m: {' '.join([p.split('/')[1] for p in dem_s3_paths])}"
cfg['product_group']['output_dir'] = str(data_dir_path/'ISCE3_RTC')
cfg['product_group']['save_mosaics'] = True
cfg['product_group']['product_path'] = '.'
cfg['product_group']['scratch_path'] = str(data_dir_path/'scratch_dir')
cfg['product_group']['product_id'] = f'OPERA_L2_RTC-S1_{scene}'
cfg['processing']['polarization'] = 'dual-pol'

In [ ]:
config

In [ ]:
with open(rtc_config_path, 'w') as f:
    yaml.dump(config, f)

## Run rtc_s1_single_job.py

- Open rtc_s1.yaml and adjust any parameters you like

In [ ]:
zip_path = data_dir_path/f"{scene}.zip"

if not zip_path.exists():
    results = asf_search.granule_search(scene)
    results[0].download(data_dir_path, session=user_pass_session)

In [ ]:
! ~/.local/envs/isce3_rtc/bin/rtc_s1_single_job.py $rtc_config_path